In [54]:
import pandas as pd
import numpy as np
import warnings
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Suppress warnings
warnings.filterwarnings("ignore")

In [55]:
# Read the data file
names = ['id','age', 'gender', 'education', 'country',
'ethnicity', 'neuroticism', 'extraversion', 'openness_to_experience',
'agreeableness','conscientiousness', 'impulsiveness', 'sensation', 'alcohol', 'amphet',
'amyl', 'benzos', 'caff', 'cannabis', 'choc', 'coke', 'crack', 'ecstasy',
'heroin', 'ketamine', 'legalh', 'lsd', 'meth', 'mushrooms',
'nicotine', 'semer', 'vsa']
df = pd.read_csv('drug_consumption.data', names=names)

In [56]:
df.head()

,id,age,gender,education,country,ethnicity,neuroticism,extraversion,openness_to_experience,agreeableness,...,ecstasy,heroin,ketamine,legalh,lsd,meth,mushrooms,nicotine,semer,vsa
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [57]:
# Filter the columns
df_filtered = df[['id','age', 'gender', 'education',
          'neuroticism', 'extraversion', 
          'openness_to_experience', 'agreeableness',
          'conscientiousness', 'impulsiveness', 'sensation',
          'cannabis']]

In [58]:
df_filtered.head()

,id,age,gender,education,neuroticism,extraversion,openness_to_experience,agreeableness,conscientiousness,impulsiveness,sensation,cannabis
0,1,0.49788,0.48246,-0.05921,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,CL0
1,2,-0.07854,-0.48246,1.98437,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,CL4
2,3,0.49788,-0.48246,-0.05921,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,CL3
3,4,-0.95197,0.48246,1.16365,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,CL2
4,5,0.49788,0.48246,1.98437,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,CL3


In [59]:
# # Replace Cannabis list with original classification
# df_filtered.replace({
#     'CL0' : 'Never Used', 
#     'CL1' : 'Used over a Decade Ago',
#     'CL2' : 'Used in Last Decade',
#     'CL3' : 'Used in Last Year',
#     'CL4' : 'Used in Last Month',
#     'CL5' : 'Used in Last Week',
#     'CL6' : 'Used in Last Day'})

In [60]:
# Replace and combine the cannabis result into 0 = never used and 1 = used -past or present-
df_cannabis = df_filtered.replace({
    'CL0' : 0, 
    'CL1' : 1,
    'CL2' : 1,
    'CL3' : 1,
    'CL4' : 1,
    'CL5' : 1,
    'CL6' : 1})

df_cannabis.head()

,id,age,gender,education,neuroticism,extraversion,openness_to_experience,agreeableness,conscientiousness,impulsiveness,sensation,cannabis
0,1,0.49788,0.48246,-0.05921,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,0
1,2,-0.07854,-0.48246,1.98437,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,1
2,3,0.49788,-0.48246,-0.05921,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,1
3,4,-0.95197,0.48246,1.16365,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,1
4,5,0.49788,0.48246,1.98437,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,1


In [61]:
# Check for nulls
df_cannabis.isnull().sum().head()

id             0
age            0
gender         0
education      0
neuroticism    0
dtype: int64

In [62]:
list(df_cannabis)
df.shape

(1885, 32)

In [63]:
from sklearn.model_selection import train_test_split

target = df_cannabis["cannabis"]

data = df_cannabis.drop(columns=["cannabis","id"])
feature_names = data.columns

In [64]:
data.head()

,age,gender,education,neuroticism,extraversion,openness_to_experience,agreeableness,conscientiousness,impulsiveness,sensation
0,0.49788,0.48246,-0.05921,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084
1,-0.07854,-0.48246,1.98437,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575
2,0.49788,-0.48246,-0.05921,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148
3,-0.95197,0.48246,1.16365,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084
4,0.49788,0.48246,1.98437,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575


In [65]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=1, stratify=target)

In [66]:
X_train.head()

,age,gender,education,neuroticism,extraversion,openness_to_experience,agreeableness,conscientiousness,impulsiveness,sensation
785,-0.95197,-0.48246,-0.05921,-1.69163,1.28610,-0.58331,-0.60633,-1.13788,1.86203,0.76540
1341,-0.95197,-0.48246,-1.22751,1.37297,-1.23177,-1.11902,-2.53830,0.93949,-1.37983,-0.21575
53,1.09449,-0.48246,1.16365,-1.43907,-0.15487,1.06238,0.28783,0.25953,-0.21712,-0.21575
974,-0.95197,-0.48246,-0.61113,-0.58016,1.11406,0.72330,0.59042,-0.00665,-0.21712,-0.52593
1209,-0.95197,-0.48246,-0.61113,-0.24649,-1.09207,0.29338,-0.45321,-1.01450,-1.37983,0.76540


In [67]:
# Scale the data using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [68]:
# Create SVC model
from sklearn.svm import SVC
SVCmodel = SVC(kernel='linear')
SVCmodel.fit(X_train_scaled, y_train)

print(f"Training Data Score: {SVCmodel.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {SVCmodel.score(X_test_scaled, y_test)}")

Training Data Score: 0.7806086341118188
Testing Data Score: 0.7817796610169492


In [69]:
# Create the GridSearchCV model for SVC model
from sklearn.model_selection import GridSearchCV
svc_param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01],
              'kernel': ['linear']}
svc_grid = GridSearchCV(SVCmodel, svc_param_grid, verbose=3)

In [70]:
# Fit the model using the grid search estimator
svc_grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV]  C=1, gamma=0.0001, kernel=linear, score=0.7796610169491526, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV]  C=1, gamma=0.0001, kernel=linear, score=0.7813163481953291, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=linear ................................
[CV]  C=1, gamma=0.0001, kernel=linear, score=0.7808510638297872, total=   0.0s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV]  C=1, gamma=0.001, kernel=linear, score=0.7796610169491526, total=   0.0s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV]  C=1, gamma=0.001, kernel=linear, score=0.7813163481953291, total=   0.0s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV]  C=1, gamma=0.001, kernel=linear, score=0.7808510638297872, total=   0.0s
[CV] C=1, gamma=0.01, kernel=linear .

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=5, gamma=0.001, kernel=linear, score=0.7813163481953291, total=   0.0s
[CV] C=5, gamma=0.001, kernel=linear .................................
[CV]  C=5, gamma=0.001, kernel=linear, score=0.7808510638297872, total=   0.0s
[CV] C=5, gamma=0.01, kernel=linear ..................................
[CV]  C=5, gamma=0.01, kernel=linear, score=0.7860169491525424, total=   0.0s
[CV] C=5, gamma=0.01, kernel=linear ..................................
[CV]  C=5, gamma=0.01, kernel=linear, score=0.7813163481953291, total=   0.0s
[CV] C=5, gamma=0.01, kernel=linear ..................................
[CV]  C=5, gamma=0.01, kernel=linear, score=0.7808510638297872, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV]  C=10, gamma=0.0001, kernel=linear, score=0.7817796610169492, total=   0.0s
[CV] C=10, gamma=0.0001, kernel=linear ...............................
[CV]  C=10, gamma=0.0001, kernel=linear, score=0.7813163481953291, total=   0.0s
[CV] C=10, gamma=0.0

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    0.3s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01], 'kernel': ['linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [71]:
# Print scores for SVC
print(svc_grid.best_params_)
print(svc_grid.best_score_)

{'C': 5, 'gamma': 0.0001, 'kernel': 'linear'}
0.7827317763623496


In [72]:
# Logistic Regression model
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression()

# Train the model
model_log.fit(X_train_scaled, y_train)

# Print scores
print(f"Training Data Score: {model_log.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model_log.score(X_test_scaled, y_test)}")

Training Data Score: 0.7983014861995754
Testing Data Score: 0.826271186440678


In [73]:
# Create the GridSearchCV model for logistic regression
from sklearn.model_selection import GridSearchCV

logistic_param_grid = {"penalty": ['l1', 'l2'],
              "C": np.logspace(0, 4, 10)}
logistic_grid = GridSearchCV(model_log, logistic_param_grid, cv=5, verbose=3)

In [74]:
# Fit the model using the grid search estimator
logistic_grid.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...... C=1.0, penalty=l1, score=0.7773851590106007, total=   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ....... C=1.0, penalty=l1, score=0.773851590106007, total=   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...... C=1.0, penalty=l1, score=0.8021201413427562, total=   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...... C=1.0, penalty=l1, score=0.8049645390070922, total=   0.0s
[CV] C=1.0, penalty=l1 ...............................................
[CV] ...... C=1.0, penalty=l1, score=0.8014184397163121, total=   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ...... C=1.0, penalty=l2, score=0.7773851590106007, total=   0.0s
[CV] C=1.0, penalty=l2 ...............................................
[CV] ....... C=

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  C=59.94842503189409, penalty=l1, score=0.8014184397163121, total=   0.0s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV]  C=59.94842503189409, penalty=l2, score=0.7773851590106007, total=   0.0s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV]  C=59.94842503189409, penalty=l2, score=0.773851590106007, total=   0.0s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV]  C=59.94842503189409, penalty=l2, score=0.8021201413427562, total=   0.0s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV]  C=59.94842503189409, penalty=l2, score=0.8014184397163121, total=   0.0s
[CV] C=59.94842503189409, penalty=l2 .................................
[CV]  C=59.94842503189409, penalty=l2, score=0.8014184397163121, total=   0.0s
[CV] C=166.81005372000593, penalty=l1 ................................
[CV]  C=166.81005372000593, penalty=l1, score=0.7773851590106007, total=   0.0s
[CV] C=166.8100537200

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([1.00000e+00, 2.78256e+00, 7.74264e+00, 2.15443e+01, 5.99484e+01,
       1.66810e+02, 4.64159e+02, 1.29155e+03, 3.59381e+03, 1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [75]:
# Print scores for Logistic Regression
print(logistic_grid.best_params_)
print(logistic_grid.best_score_)

{'C': 1.0, 'penalty': 'l1'}
0.7919320594479831


In [76]:
# Linear regression model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

# Train the model
linear_model.fit(X_train_scaled, y_train)

# Print scores
print(f"Training Data Score: {linear_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {linear_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.21024063495528178
Testing Data Score: 0.2615306422104138


In [77]:
# Create the GridSearchCV model for Linear Regression
from sklearn.model_selection import GridSearchCV
param_grid = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
linear_grid = GridSearchCV(linear_model, param_grid, verbose=3)

In [78]:
# Fit the model using the grid search estimator
linear_grid.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.15876311615648842, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.2133701207679224, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.22444001020956117, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.1587631161564883, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.21337012076792208, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=T

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.0s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_intercept': [True, False], 'normalize': [True, False], 'copy_X': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [79]:
# Print LinearRegression best params and best score
print(linear_grid.best_params_)
print(linear_grid.best_score_)

{'copy_X': True, 'fit_intercept': True, 'normalize': True}
0.19885774904465733


In [80]:
# Ridge model
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train)

print(f"Training Data Score: {ridge.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {ridge.score(X_test_scaled, y_test)}")

Training Data Score: 0.21024062654868914
Testing Data Score: 0.2615335927964306


In [81]:
# Create the GridSearchCV model for Ridge
from sklearn.model_selection import GridSearchCV

param_grid_ridge = {'alpha': [0.001, 0.01, 0.1]}
ridge_grid = GridSearchCV(ridge, param_grid_ridge, n_jobs=1, cv=10)

In [82]:
# Fit the model using the grid search estimator
ridge_grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'alpha': [0.001, 0.01, 0.1]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [83]:
# Print scores for Ridge
print(ridge_grid.best_params_)
print(ridge_grid.best_score_)

{'alpha': 0.1}
0.1944766589154363


In [84]:
# ElasticNet model
from sklearn.linear_model import ElasticNet

# Create and train the model
elastic_model = ElasticNet(alpha=.01).fit(X_train_scaled, y_train)

# Print scores
print(f"Training Data Score: {elastic_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {elastic_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.18138962033548223
Testing Data Score: 0.24513330727020374


In [85]:
# Create the GridSearchCV model for ElasticNet
from sklearn.model_selection import GridSearchCV
elastic_param_grid = {
                     "max_iter": [1, 5, 10],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0, 1.0, 0.1)
                     }
elastic_grid = GridSearchCV(elastic_model, elastic_param_grid, verbose=3, scoring='r2', cv=10)

In [86]:
# Fit the model using the grid search estimator
elastic_grid.fit(X_train_scaled, y_train)

Fitting 10 folds for each of 210 candidates, totalling 2100 fits
[CV] alpha=0.0001, l1_ratio=0.0, max_iter=1 ..........................
[CV]  alpha=0.0001, l1_ratio=0.0, max_iter=1, score=0.1728059462736331, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.0, max_iter=1 ..........................
[CV]  alpha=0.0001, l1_ratio=0.0, max_iter=1, score=0.20326194528694996, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.0, max_iter=1 ..........................
[CV]  alpha=0.0001, l1_ratio=0.0, max_iter=1, score=-0.02914411950260276, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.0, max_iter=1 ..........................
[CV]  alpha=0.0001, l1_ratio=0.0, max_iter=1, score=0.1813390607195482, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.0, max_iter=1 ..........................
[CV]  alpha=0.0001, l1_ratio=0.0, max_iter=1, score=0.03457201113708053, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.0, max_iter=1 ..........................
[CV]  alpha=0.0001, l1_ratio=0.0, max_iter=1, score=0.22720252411555708, to

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5 ..........
[CV]  alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5, score=0.21991436534422082, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5 ..........
[CV]  alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5, score=0.06160175284847347, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5 ..........
[CV]  alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5, score=0.20964175061982648, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5 ..........
[CV]  alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5, score=0.09831177428020144, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5 ..........
[CV]  alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5, score=0.25818986450386316, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=5 ..........
[CV]  alpha=0.0001, l1_ratio=0.30000000000000004, max_iter=

[CV]  alpha=0.0001, l1_ratio=0.6000000000000001, max_iter=10, score=0.23802489711893482, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.6000000000000001, max_iter=10 ..........
[CV]  alpha=0.0001, l1_ratio=0.6000000000000001, max_iter=10, score=0.24553192884370745, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1 ...........
[CV]  alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1, score=0.17287836902194986, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1 ...........
[CV]  alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1, score=0.20365234734881166, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1 ...........
[CV]  alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1, score=-0.028154587562382094, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1 ...........
[CV]  alpha=0.0001, l1_ratio=0.7000000000000001, max_iter=1, score=0.18135607305784796, total=   0.0s
[CV] alpha=0.0001, l1_ratio=0

[CV]  alpha=0.001, l1_ratio=0.0, max_iter=10, score=0.20068765438696445, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.0, max_iter=10 ..........................
[CV]  alpha=0.001, l1_ratio=0.0, max_iter=10, score=0.21825934766039579, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.0, max_iter=10 ..........................
[CV]  alpha=0.001, l1_ratio=0.0, max_iter=10, score=0.06382706725096365, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.0, max_iter=10 ..........................
[CV]  alpha=0.001, l1_ratio=0.0, max_iter=10, score=0.2103990713970203, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.0, max_iter=10 ..........................
[CV]  alpha=0.001, l1_ratio=0.0, max_iter=10, score=0.10120322861247888, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.0, max_iter=10 ..........................
[CV]  alpha=0.001, l1_ratio=0.0, max_iter=10, score=0.25753433552181837, total=   0.0s
[CV] alpha=0.001, l1_ratio=0.0, max_iter=10 ..........................
[CV]  alpha=0.001, l1_ratio=0.0, max_iter=10, score=0

...

[CV]  alpha=100, l1_ratio=0.1, max_iter=10, score=-0.014984212153669185, total=   0.0s
[CV] alpha=100, l1_ratio=0.1, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.1, max_iter=10, score=-1.5676073659154355e-06, total=   0.0s
[CV] alpha=100, l1_ratio=0.1, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.1, max_iter=10, score=-0.001421310682522492, total=   0.0s
[CV] alpha=100, l1_ratio=0.1, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.1, max_iter=10, score=-0.0031881590450546504, total=   0.0s
[CV] alpha=100, l1_ratio=0.1, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.1, max_iter=10, score=-0.005498344330746585, total=   0.0s
[CV] alpha=100, l1_ratio=0.1, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.1, max_iter=10, score=-1.5676073659154355e-06, total=   0.0s
[CV] alpha=100, l1_ratio=0.2, max_iter=1 .............................
[CV]  alpha=100, l1_ratio=0.2, max_iter=1, scor

[CV]  alpha=100, l1_ratio=0.5, max_iter=5, score=-0.001421310682522492, total=   0.0s
[CV] alpha=100, l1_ratio=0.5, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.5, max_iter=5, score=-0.0031881590450546504, total=   0.0s
[CV] alpha=100, l1_ratio=0.5, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.5, max_iter=5, score=-0.005498344330746585, total=   0.0s
[CV] alpha=100, l1_ratio=0.5, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.5, max_iter=5, score=-1.5676073659154355e-06, total=   0.0s
[CV] alpha=100, l1_ratio=0.5, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.5, max_iter=10, score=-0.0011714307479053776, total=   0.0s
[CV] alpha=100, l1_ratio=0.5, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.5, max_iter=10, score=-0.00025155247503194644, total=   0.0s
[CV] alpha=100, l1_ratio=0.5, max_iter=10 ............................
[CV]  alpha=100, l1_ratio=0.5, max_iter=10, score=

[CV]  alpha=100, l1_ratio=0.9, max_iter=1, score=-0.005498344330746585, total=   0.0s
[CV] alpha=100, l1_ratio=0.9, max_iter=1 .............................
[CV]  alpha=100, l1_ratio=0.9, max_iter=1, score=-1.5676073659154355e-06, total=   0.0s
[CV] alpha=100, l1_ratio=0.9, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.9, max_iter=5, score=-0.0011714307479053776, total=   0.0s
[CV] alpha=100, l1_ratio=0.9, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.9, max_iter=5, score=-0.00025155247503194644, total=   0.0s
[CV] alpha=100, l1_ratio=0.9, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.9, max_iter=5, score=-0.0068671338646373314, total=   0.0s
[CV] alpha=100, l1_ratio=0.9, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.9, max_iter=5, score=-0.0014775979230818859, total=   0.0s
[CV] alpha=100, l1_ratio=0.9, max_iter=5 .............................
[CV]  alpha=100, l1_ratio=0.9, max_iter=5, score=-0

[Parallel(n_jobs=1)]: Done 2100 out of 2100 | elapsed:    3.5s finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_iter': [1, 5, 10], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=3)

In [87]:
# Print scores for ElasticNet
print(elastic_grid.best_params_)
print(elastic_grid.best_score_)

{'alpha': 0.001, 'l1_ratio': 0.6000000000000001, 'max_iter': 10}
0.19546906734076158


In [88]:
# Compare scores

# Print LinearRegression best params and best score
print("LinearRegression Scores")
print(linear_grid.best_params_)
print(linear_grid.best_score_)
print("-------------------------")

# Print scores for logistic regression
print("LogisticRegression Scores")
print(logistic_grid.best_params_)
print(logistic_grid.best_score_)
print("-------------------------")

# Print scores for SVC
print("SVC Scores")
print(svc_grid.best_params_)
print(svc_grid.best_score_)
print("-------------------------")

# Print scores for Ridge
print("Ridge Scores")
print(ridge_grid.best_params_)
print(ridge_grid.best_score_)
print("-------------------------")

# Print scores for ElasticNet
print("ElasticNet Scores")
print(elastic_grid.best_params_)
print(elastic_grid.best_score_)

LinearRegression Scores
{'copy_X': True, 'fit_intercept': True, 'normalize': True}
0.19885774904465733
-------------------------
LogisticRegression Scores
{'C': 1.0, 'penalty': 'l1'}
0.7919320594479831
-------------------------
SVC Scores
{'C': 5, 'gamma': 0.0001, 'kernel': 'linear'}
0.7827317763623496
-------------------------
Ridge Scores
{'alpha': 0.1}
0.1944766589154363
-------------------------
ElasticNet Scores
{'alpha': 0.001, 'l1_ratio': 0.6000000000000001, 'max_iter': 10}
0.19546906734076158


In [89]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [90]:
# Deep Learning Model

# Create a deep neural network
deep_model = Sequential()
deep_model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
deep_model.add(Dense(units=64, activation='relu'))
deep_model.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))

In [91]:
# Compile the model
deep_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [92]:
deep_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               1408      
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 130       
Total params: 9,794
Trainable params: 9,794
Non-trainable params: 0
_________________________________________________________________


In [93]:
# Fit the model to the training data
deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 0s - loss: 0.5333 - acc: 0.7410
Epoch 2/100
 - 0s - loss: 0.4481 - acc: 0.7870
Epoch 3/100
 - 0s - loss: 0.4319 - acc: 0.7948
Epoch 4/100
 - 0s - loss: 0.4185 - acc: 0.7955
Epoch 5/100
 - 0s - loss: 0.4176 - acc: 0.7997
Epoch 6/100
 - 0s - loss: 0.4127 - acc: 0.8025
Epoch 7/100
 - 0s - loss: 0.4104 - acc: 0.7969
Epoch 8/100
 - 0s - loss: 0.4078 - acc: 0.7969
Epoch 9/100
 - 0s - loss: 0.4065 - acc: 0.8018
Epoch 10/100
 - 0s - loss: 0.4085 - acc: 0.8047
Epoch 11/100
 - 0s - loss: 0.4020 - acc: 0.7983
Epoch 12/100
 - 0s - loss: 0.4006 - acc: 0.8011
Epoch 13/100
 - 0s - loss: 0.3992 - acc: 0.8139
Epoch 14/100
 - 0s - loss: 0.4024 - acc: 0.8047
Epoch 15/100
 - 0s - loss: 0.3975 - acc: 0.7976
Epoch 16/100
 - 0s - loss: 0.3993 - acc: 0.8033
Epoch 17/100
 - 0s - loss: 0.3957 - acc: 0.8082
Epoch 18/100
 - 0s - loss: 0.3946 - acc: 0.8068
Epoch 19/100
 - 0s - loss: 0.3942 - acc: 0.8047
Epoch 20/100
 - 0s - loss: 0.3947 - acc: 0.8096
Epoch 21/100
 - 0s - loss: 0.3949 - acc: 0.8040
E

In [94]:
deep_model_loss, deep_model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Deep Neural Network - Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}")

 - 0s - loss: 0.3968 - acc: 0.8157
Deep Neural Network - Loss: 0.3967736328052262, Accuracy: 0.8156779408454895


In [95]:
# Deep Learning Model V2

# Create a deep neural network
deep_model2 = Sequential()
deep_model2.add(Dense(units=128, activation='tanh', input_dim=X_train.shape[1]))
deep_model2.add(Dense(units=64, activation='tanh'))
deep_model2.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))

In [96]:
# Compile the model
deep_model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [97]:
deep_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 128)               1408      
_________________________________________________________________
dense_15 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 130       
Total params: 9,794
Trainable params: 9,794
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Fit the model to the training data
deep_model2.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 0s - loss: 0.4666 - acc: 0.7849
Epoch 2/100
 - 0s - loss: 0.4320 - acc: 0.7856
Epoch 3/100
 - 0s - loss: 0.4211 - acc: 0.7933
Epoch 4/100
 - 0s - loss: 0.4229 - acc: 0.7941
Epoch 5/100
 - 0s - loss: 0.4204 - acc: 0.7919
Epoch 6/100
 - 0s - loss: 0.4184 - acc: 0.7941
Epoch 7/100
 - 0s - loss: 0.4205 - acc: 0.7877
Epoch 8/100
 - 0s - loss: 0.4133 - acc: 0.7948
Epoch 9/100
 - 0s - loss: 0.4156 - acc: 0.7891
Epoch 10/100
 - 0s - loss: 0.4186 - acc: 0.7969
Epoch 11/100
 - 0s - loss: 0.4150 - acc: 0.7898
Epoch 12/100
 - 0s - loss: 0.4160 - acc: 0.7891
Epoch 13/100
 - 0s - loss: 0.4133 - acc: 0.7962
Epoch 14/100
 - 0s - loss: 0.4164 - acc: 0.7941
Epoch 15/100
 - 0s - loss: 0.4115 - acc: 0.7969
Epoch 16/100
 - 0s - loss: 0.4156 - acc: 0.7955
Epoch 17/100
 - 0s - loss: 0.4132 - acc: 0.7969
Epoch 18/100
 - 0s - loss: 0.4156 - acc: 0.7955
Epoch 19/100
 - 0s - loss: 0.4168 - acc: 0.7976
Epoch 20/100
 - 0s - loss: 0.4097 - acc: 0.7933
Epoch 21/100
 - 0s - loss: 0.4086 - acc: 0.8004
E

In [99]:
deep_model2_loss, deep_model2_accuracy = deep_model2.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Deep Neural Network V2 - Loss: {deep_model2_loss}, Accuracy: {deep_model2_accuracy}")

 - 0s - loss: 0.3667 - acc: 0.8242
Deep Neural Network V2 - Loss: 0.36671625507079947, Accuracy: 0.8241525292396545


In [100]:
# Deep Learning Model V3
# Create a deep neural network
deep_model3 = Sequential()
deep_model3.add(Dense(units=500, activation='tanh', input_dim=X_train.shape[1]))
deep_model3.add(Dense(units=500, activation='tanh'))
deep_model3.add(Dense(units=500, activation='tanh'))
deep_model3.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))

In [101]:
# Compile the model
deep_model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [102]:
deep_model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 500)               5500      
_________________________________________________________________
dense_18 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_19 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 1002      
Total params: 507,502
Trainable params: 507,502
Non-trainable params: 0
_________________________________________________________________


In [103]:
# Fit the model to the training data
deep_model3.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 0.4664 - acc: 0.7714
Epoch 2/100
 - 0s - loss: 0.4439 - acc: 0.7891
Epoch 3/100
 - 0s - loss: 0.4303 - acc: 0.7962
Epoch 4/100
 - 0s - loss: 0.4436 - acc: 0.7933
Epoch 5/100
 - 0s - loss: 0.4231 - acc: 0.7898
Epoch 6/100
 - 0s - loss: 0.4210 - acc: 0.7926
Epoch 7/100
 - 0s - loss: 0.4272 - acc: 0.7969
Epoch 8/100
 - 0s - loss: 0.4397 - acc: 0.7891
Epoch 9/100
 - 0s - loss: 0.4237 - acc: 0.8033
Epoch 10/100
 - 0s - loss: 0.4198 - acc: 0.7948
Epoch 11/100
 - 0s - loss: 0.4156 - acc: 0.7990
Epoch 12/100
 - 0s - loss: 0.4177 - acc: 0.7919
Epoch 13/100
 - 0s - loss: 0.4151 - acc: 0.8011
Epoch 14/100
 - 0s - loss: 0.4214 - acc: 0.7933
Epoch 15/100
 - 0s - loss: 0.4169 - acc: 0.7870
Epoch 16/100
 - 0s - loss: 0.4440 - acc: 0.7863
Epoch 17/100
 - 0s - loss: 0.4201 - acc: 0.7905
Epoch 18/100
 - 0s - loss: 0.4182 - acc: 0.7926
Epoch 19/100
 - 0s - loss: 0.4219 - acc: 0.7884
Epoch 20/100
 - 0s - loss: 0.4174 - acc: 0.7969
Epoch 21/100
 - 0s - loss: 0.4180 - acc: 0.7891
E

In [104]:
deep_model3_loss, deep_model3_accuracy = deep_model3.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Deep Neural Network V2 - Loss: {deep_model3_loss}, Accuracy: {deep_model3_accuracy}")

 - 0s - loss: 0.3712 - acc: 0.8242
Deep Neural Network V2 - Loss: 0.37115335262427895, Accuracy: 0.8241525292396545
